In [82]:
!pip install spotipy

#구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np
import time
import os

In [84]:
class mySpotify:
    def __init__(self):
        # API 클라이언트 등록
        cid = '4598e66debe040afa4e5e77d7f33ee1d'
        secret = '604c1a9fa94c4a939cdb376b1ddede2a'

        # 보안 등록 후 spotify 객체 생성
        client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
        self.sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    def get_track_id(self, title, artist):
      # title을 통해 곡 정보 요청
      query = title  
      track_info = self.sp.search(q=query, type='track', market = 'KR')

      # title로 검색된 곡들을 하나씩 탐색
      for currSong in track_info['tracks']['items']:
          curr_title = currSong['name']
          
          # artist가 두명 이상일 경우
          if len(currSong['artists']) > 1 :
            # artist들을 ','로 연결한 string을 생성하여 
            curr_artist = ', '.join( artist['name'] for artist in currSong['artists'] )
          
          else:
            curr_artist = currSong['artists'][0]['name']
          
          # # 일치하는 곡정보를 탐색했는지 판별
          # print('검색된 curr_title({})와 검색한 title({})의 일치여부: {}, \n검색된 curr_arist({})와 검색한 artit({})의 일치여부: {}\n'.format(curr_title, title, curr_title == title, curr_artist, artist, curr_artist == artist))
          
          # 찾고자 하는 곡 정보와 일치할 경우 필요한 response 리턴
          if (curr_title == title) and (curr_artist == artist):
            # print('탐색 완료!\n')
            title = curr_title
            artist = curr_artist
            id = currSong['id']
            img = currSong['album']['images'][0]['url']
            url = currSong['external_urls']['spotify']
            preview = currSong['preview_url']
            return [title, artist, id, img, url, preview]
          else: continue

      return None

    def get_features(self, songID):
      features = self.sp.audio_features(tracks=[songID])[0]
      if features :
        danceability = features['danceability']
        energy = features['energy']
        key = features['key']
        loudness = features['loudness']
        mode = features['mode']
        speechiness = features['speechiness']
        acousticness = features['acousticness']
        instrumentalness = features['instrumentalness']
        liveness = features['liveness']
        valence = features['valence']
        tempo = features['tempo']
        
        return (energy, danceability, valence)

      else:
        return None

In [85]:
# 사용할 감정 리스트
emotionDict = {
    1: 'SAD',
    2: 'MILD',
    3: 'NATURE',
    4: 'LOUD',
    5: 'SENTIMENTAL',
    6: 'FUNNY',
    7: 'LOVELY',
}

# 감정 선택
emotion = emotionDict[7]

# 정보를 수집할 파일 경로
WORKSPACE_ROUTE = '/content/drive/My Drive/Colab Notebooks/(AI_Project) unemployedStation/'
PLAYLIST_ROUTE = WORKSPACE_ROUTE + 'data/playlist/'+ emotion + '_PLAYLIST.csv'
TRACK_INFO_ROUTE = WORKSPACE_ROUTE + 'data/trackInfo/'+ emotion + '_TRACKINFO.csv'

playlist = pd.read_csv(PLAYLIST_ROUTE)
print('플레이리스트로 수집한 데이터의 수:', len(playlist))

플레이리스트로 수집한 데이터의 수: 175


In [86]:
# 플레이리스트 간에 중복되는 트랙 확인
display(playlist[playlist.duplicated()])

# 중복되는 트랙 제거
playlist.drop_duplicates(ignore_index=True, inplace = True)
print('\n중복되는 트랙을 제거하고 남은 데이터의 수:', len(playlist))

,title,artist
113,Fairy of Shampoo,TOMORROW X TOGETHER
115,Some,BOL4
128,Beige coat,Grizzly



중복되는 트랙을 제거하고 남은 데이터의 수: 172


In [ ]:
# 기능별로 필요한 동작을 수행하도록 설계한 스포티파이 객체
saf = mySpotify()

### search API 적용

# 필요한 정보를 담을 리스트
info_list = []

# 탐색할 플레이리스트의 레코드 수만큼 반복
for i in range(len(playlist)):
  song_title = playlist.loc[i][0]
  song_artist = playlist.loc[i][1]
  # print('탐색 중..\n->', song_title, ' | ', song_artist)
  info = saf.get_track_id(song_title, song_artist)
  if info:
    info_list.append(info)
  else:
    print('없음 -> ( title:', song_title, ' | artist:', song_artist, ')\n')
  
  if ((i+1) % 10 == 0):
    print('\n탐색 완료! (', i+1, '/', len(playlist), ')\n\n')

# search API를 통해 수집한 정보가 담긴 데이터프레임 생성
df = pd.DataFrame(info_list, columns=['TrackName','Artist','SongID', 'IMG','SongURL','Preview'])
print(len(df))


탐색 완료! ( 10 / 172 )


없음 -> ( title: Sweet  | artist: Lee Mujin )


탐색 완료! ( 20 / 172 )



탐색 완료! ( 30 / 172 )


없음 -> ( title: See You Again  | artist: Yerin Baek )

없음 -> ( title: Magic  | artist: Song Youjin )


탐색 완료! ( 40 / 172 )


없음 -> ( title: Fool  | artist: JUNIEL, Jung Yong Hwa )


탐색 완료! ( 50 / 172 )


없음 -> ( title: A Very Special One  | artist: Sugarbowl )

없음 -> ( title: Angel  | artist: NCT 127 )

없음 -> ( title: Turtle  | artist: TWICE )

없음 -> ( title: Love Letter  | artist: Golden Child )


탐색 완료! ( 60 / 172 )


없음 -> ( title: Let Me  | artist: GOT7 )


탐색 완료! ( 70 / 172 )



탐색 완료! ( 80 / 172 )


없음 -> ( title: About Now  | artist: DAY6 )

없음 -> ( title: Deep in love  | artist: DAY6 )

없음 -> ( title: I Want To Live With You  | artist: Alex Lahey )

없음 -> ( title: 그날 The Day  | artist: Jinyoung )


탐색 완료! ( 90 / 172 )



탐색 완료! ( 100 / 172 )



탐색 완료! ( 110 / 172 )



탐색 완료! ( 120 / 172 )


없음 -> ( title: Where the sea sleeps  | artist: DAY6 (Even of Day) )


탐색 완료! 

In [ ]:
### audio_features API 적용

# search API를 통해 수집된 데이터프레임에 Audio Feature 컬럼 추가
df['Energy'] = np.nan
df['Danceability'] = np.nan
df['Valence'] = np.nan

cnt = 0
# Audio Feature 수집
for i in range(len(df)):
  feature = saf.get_features(df.iloc[i]['SongID'])

  if feature:
    energy, danceability, valence = feature
    df.iloc[i,6] = energy
    df.iloc[i,7] = danceability
    df.iloc[i,8] = valence

  else:
    cnt += 1
    print('No Feature -> ( title:', song_title, ' | artist:', song_artist, ')')

  if ((i+1) % 10 == 0):
    print('\nCollected! (', i+1, '/', len(df), ')\n')

print('\nAudio Feature까지 수집 완료된 "', emotion, ' " 데이터의 수: ', len(df) - cnt)

# 완성된 테이블을 CSV로 저장
if not os.path.exists(TRACK_INFO_ROUTE):
  df.to_csv(TRACK_INFO_ROUTE, index=False, mode='w', encoding='utf-8-sig')
else:
  df.to_csv(TRACK_INFO_ROUTE, index=False, mode='a', encoding='utf-8-sig', header=False)


Collected! ( 10 / 157 )


Collected! ( 20 / 157 )


Collected! ( 30 / 157 )


Collected! ( 40 / 157 )


Collected! ( 50 / 157 )


Collected! ( 60 / 157 )


Collected! ( 70 / 157 )


Collected! ( 80 / 157 )


Collected! ( 90 / 157 )


Collected! ( 100 / 157 )


Collected! ( 110 / 157 )


Collected! ( 120 / 157 )


Collected! ( 130 / 157 )


Collected! ( 140 / 157 )


Collected! ( 150 / 157 )


Audio Feature까지 수집 완료된 " LOVELY  " 데이터의 수:  157


In [ ]:
display(df.head(4))
display(df.tail(4))

,TrackName,Artist,SongID,IMG,SongURL,Preview,Energy,Danceability,Valence
0,Some,BOL4,3jsYQw78lrxJA2ysnmOIf9,https://i.scdn.co/image/ab67616d0000b2735a779a...,https://open.spotify.com/track/3jsYQw78lrxJA2y...,https://p.scdn.co/mp3-preview/051c58da61477df7...,0.799,0.659,0.782
1,Say Yes,"Loco, Punch",27zrFrtUtWl2urlvjOn5xc,https://i.scdn.co/image/ab67616d0000b273cb9596...,https://open.spotify.com/track/27zrFrtUtWl2url...,https://p.scdn.co/mp3-preview/ce97bf17721ab769...,0.778,0.696,0.410
2,Fairy of Shampoo,TOMORROW X TOGETHER,5JXpZcc3CeQaC6vAF1iaV5,https://i.scdn.co/image/ab67616d0000b273b85b1b...,https://open.spotify.com/track/5JXpZcc3CeQaC6v...,https://p.scdn.co/mp3-preview/d8c72a71352afe4a...,0.796,0.544,0.473
3,Perhaps Love (사랑인가요) (Prod.By 박근태),"에릭남 (Eric Nam), CHEEZE (치즈)",5bN1ltT5BhVMnszmgsqGD5,https://i.scdn.co/image/ab67616d0000b2739793ec...,https://open.spotify.com/track/5bN1ltT5BhVMnsz...,https://p.scdn.co/mp3-preview/7654e525f5cbcaa3...,0.445,0.525,0.258


,TrackName,Artist,SongID,IMG,SongURL,Preview,Energy,Danceability,Valence
153,Mirage,Unchan,23z6RGYVcJOsY2qwCESHa9,https://i.scdn.co/image/ab67616d0000b273fc46cf...,https://open.spotify.com/track/23z6RGYVcJOsY2q...,https://p.scdn.co/mp3-preview/99353b587aae75bd...,0.569,0.557,0.476
154,gr8,Llwyd,0BsQtnv3ch7w64B52ik72W,https://i.scdn.co/image/ab67616d0000b2736e26a8...,https://open.spotify.com/track/0BsQtnv3ch7w64B...,https://p.scdn.co/mp3-preview/3dac9d97d425b115...,0.532,0.693,0.609
155,Still with you,SU A,4qL1uZDOmVW5sgXi1IAOic,https://i.scdn.co/image/ab67616d0000b273bb5f3c...,https://open.spotify.com/track/4qL1uZDOmVW5sgX...,https://p.scdn.co/mp3-preview/6b5452c9d4b8dbc7...,0.417,0.501,0.332
156,I Will Be By Your Side,MeloMance,7gWKxBSrDUfs4dWaneLVKm,https://i.scdn.co/image/ab67616d0000b27329ea41...,https://open.spotify.com/track/7gWKxBSrDUfs4dW...,https://p.scdn.co/mp3-preview/9669d4f4c2a58fe9...,0.757,0.665,0.321
